In [ ]:
## Importing Libraries

# Python-based Libraries
from datetime import timedelta, datetime
from pathlib import Path
import os

# Data Analysis Libraries
import pandas as pd

# Graph plotting libraries

    
# Filtering Data before Plotting


In [ ]:
class Drifters:
    
    def __init__(self, name:str):
        self.name = name
        self.data = {}
        self.data_info = {}
        
    def read_data(self, path):
        # Define Possible encodings
        encodings = ["UTF-8", "UTF-16 LE"]

        # Check if path is a filepath or directory path
        if Path(path).is_file(): 
            # Get file name from path
            path = path.replace
            filename = path.split('/')[-1].split('.')[-2] 
            # Define a variable to track file decoding
            encoding_gotten = False
            for encoding_value in encodings:
                try:
                    data_value = pd.read_csv(path, encoding=encoding_value)
                    # Catch special errors whereby the encoding is wrong but the data is read.
                    if "Unnamed: 1" in data_value.columns: 
                        continue
                    encoding_gotten = True
                    break # Stop trying if the options had worked
                except:
                    pass # Continue trying if the option didn't work

            if encoding_gotten == True:
                data_to_keep = [['DeviceDateTime','Latitude','Longitude'], 
                            ['Position time (UTC)', 'Latitude (°)', 'Longitude (°)'],
                            ]
                for dtk_val in data_to_keep:
                    try:
                        # Select Data to Keep
                        data_value = data_value[dtk_val]
                        # Rename Header Columns
                        data_value.columns = ['DateTime','Latitude','Longitude']
                        # Convert DateTime
                        data_value['DateTime'] = pd.to_datetime(data_value['DateTime'])
                        # Store DF in the object instance 'data'
                        self.data[filename] = data_value
                        
                    except:
                        pass
            else:
                print(f"Encoding error. The encoding for {filename} data should be checked.")

        elif Path(path).is_dir():    
        # 1. Read the files in the given path
            file_names = os.listdir(path)

            # 2. Read each file into pandas dataframe
            for file in file_names:
                filepath = path+'/'+file
                filename = file.split('.')[-2] 
                # Define a variable to track file decoding
                encoding_gotten = False
                for encoding_value in encodings:
                    try:
                        data_value = pd.read_csv(filepath, encoding=encoding_value)
                        # Catch special errors whereby the encoding is wrong but the data is read.
                        if "Unnamed: 1" in data_value.columns: 
                            continue
                        encoding_gotten = True
                        break # Stop trying if the options had worked
                    except:
                        pass # Continue trying if the option didn't work

                # Store the data in the "data" dictionary
                if encoding_gotten == True:
                    data_to_keep = [['DeviceDateTime','Latitude','Longitude'], 
                               ['Position time (UTC)', 'Latitude (°)', 'Longitude (°)'],
                                ]
                    for dtk_val in data_to_keep:
                        try:
                            # Select Data to Keep
                            data_value = data_value[dtk_val]
                            # Rename Header Columns
                            data_value.columns = ['DateTime','Latitude','Longitude']
                            # Convert DateTime
                            data_value['DateTime'] = pd.to_datetime(data_value['DateTime'])
                            # Store DF in the object instance 'data'
                            self.data[filename] = data_value
                        
                        except:
                            pass

                else:
                    print(f"Encoding error. The encoding for {filepath} data should be checked.")
    
    def read_logsheet(self, file_path):
        
        self.logsheet = pd.read_csv(file_path)
        
        self.logsheet["DepDateTime"] = self.logsheet.apply(lambda x: x["DepDate"] + " " + x["DepTime"])
        self.logsheet["RecovDateTime"] = self.logsheet.apply(lambda x: x["RecovDate"] + " " + x["RecovTime"])
        
        self.logsheet.drop(columns=["DepDate", "DepTime", "RecovDate", "RecovTime"], inplace=True)
        
        self.logsheet["DepDateTime"] = pd.to_datetime(self.logsheet["DepDateTime"])
        self.logsheet["RecovDateTime"] = pd.to_datetime(self.logsheet["RecovDateTime"])
        
        for key in self.data.keys():
            station = self.logsheet["Station"][self.logsheet["Name"] == float(key)].values
            drog_depth = self.logsheet["DrogDepth"][self.logsheet["Name"] == float(key)].values
            
            
            dep_datetime = self.logsheet["DepDateTime"][self.logsheet["Name"] == float(key)].values
            recov_datetime = self.logsheet["RecovDateTime"][self.logsheet["Name"] == float(key)].values
            
            dep_long = self.logsheet["DepLong"][self.logsheet["Name"] == float(key)].values
            dep_lat = self.logsheet["DepLat"][self.logsheet["Name"] == float(key)].values
            
            recov_long = self.logsheet["RecovLong"][self.logsheet["Name"] == float(key)].values
            recov_lat = self.logsheet["RecovLat"][self.logsheet["Name"] == float(key)].values
            
            
            info = {"Station":station, "DrogDepth":drog_depth, "DepDateTime":dep_datetime, "RecovDateTime":recov_datetime,
                    "DepLong":dep_long, "DepLat":dep_lat, "RecovLong":recov_long, "RecovLat":recov_lat}
            
            self.data_info[key] = info
    
    def time_shift(self, shift_amount:float = 1):
        for key in self.data.keys():
            self.data[key]["DateTime"] = self.data[key]["DateTime"] + timedelta(hours = shift_amount)
    
    def extract_data(self):
        pass
            
    
    def compute_velocity(self):
        pass
    
    def plot_trajectory(self):
        pass
    

In [ ]:
# Testing code
